# GPTモデルを微調整して簡単な日本語ニュース文章を要約(自習用)

In [134]:
!pip install transformers torch datasets --quiet

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

In [135]:
# --- モデル & トークナイザー準備 ---
model_name = "rinna/japanese-gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [136]:
# PADトークン設定（警告回避）
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [137]:
# --- 訓練用サンプル文章（短めニュース） ---
texts = [
    """文部科学省は2026年度の予算で科学技術分野に9863億円を計上した。
    科学研究費助成事業（科研費）は101億円増の2479億円で、基礎研究支援を強化する方針である。
    また、有人月探査「アルテミス計画」関連の予算も増額され、大学や企業による研究開発が活発化する見込みである。""",

    """東京では先週末、大雪の影響で交通機関に大きな遅れが発生した。
    鉄道やバスは一部運休となり、通勤・通学に大幅な影響が出た。
    道路の渋滞や滑りやすい路面により、事故防止の呼びかけも行われた。
    市民生活や物流にも影響が広がり、交通当局は注意を呼びかけた。""",

    """人気のAI家電が市場に登場した。
    音声操作や自動清掃機能を搭載し、家庭の利便性向上が期待されている。
    一部製品ではAIがユーザーの生活習慣を学習し、自動で家事の最適化を行う機能も搭載されている。"""
]

dataset = Dataset.from_dict({"text": texts})

In [138]:
# --- トークナイズ ---
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [139]:
# --- データコラトル ---
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [140]:
# --- 訓練設定 ---
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    report_to=[],  # wandb 無効化
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipython-input-550332095.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [141]:
# --- 微調整開始 ---
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 3}.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=4, training_loss=2.7323150634765625, metrics={'train_runtime': 258.1189, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.015, 'total_flos': 1393051041792.0, 'train_loss': 2.7323150634765625, 'epoch': 2.0})

In [142]:
# --- 要約生成関数 ---
def generate_summary(text, max_length=120, num_beams=5):
    prompt = f"次の文章を一文で簡潔にまとめてください:\n\n{text}\n\n要約:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=num_beams,
        no_repeat_ngram_size=2,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [143]:
# --- サンプル文章の要約 ---
sample_text = """
東京では先週末、大雪の影響で交通機関に大きな遅れが発生した。
鉄道やバスは一部運休となり、通勤・通学に大幅な影響が出た。
道路の渋滞や滑りやすい路面により、事故防止の呼びかけも行われた。
市民生活や物流にも影響が広がり、交通当局は注意を呼びかけた。
"""

summary = generate_summary(sample_text)
print("=== 要約結果 ===")
print(summary)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


=== 要約結果 ===
次の文章を一文で簡潔にまとめてください: 東京では先週末、大雪の影響で交通機関に大きな遅れが発生した。 鉄道やバスは一部運休となり、通勤・通学に大幅な影響が出た。 道路の渋滞や滑りやすい路面により、事故防止の呼びかけも行われた。 市民生活や物流にも影響が広がり、交通当局は注意を呼びかけた。 要約:先週の大雪による交通機関の混乱は、首都圏の交通網にも大きな影響をもたらした。 この影響で、首都高速道路や首都高速神奈川1号横羽線などの一部区間が通行止めとなった。
